In [1]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [2]:
from openai import OpenAI

client = OpenAI() # defautls to getting the key using os.environ.get("OPENAI_API_KEY")